In [1]:
from comet_ml import Experiment
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
import torch
from torchsummary import summary
from utils import _get_accuracy, fsp
torch.cuda.set_device(0)
torch.manual_seed(1)
torch.cuda.manual_seed(1)

# stage should be in 0 to 5 (5 for classifier stage)
hyper_params = {
    "dataset": 'imagenette',
    "stage": 0,
    "repeated": 0,
    "num_classes": 10,
    "batch_size": 64,
    "num_epochs": 100,
    "learning_rate": 1e-4
}

In [2]:
path = untar_data(URLs.IMAGENETTE)

In [3]:
tfms = get_transforms(do_flip=False)
data = ImageDataBunch.from_folder(path, train = 'train', valid = 'val', bs = hyper_params["batch_size"], size = 224, ds_tfms = tfms).normalize(imagenet_stats)

In [4]:
learn = cnn_learner(data, models.resnet34, metrics = accuracy)
learn = learn.load('resnet34_imagenette_bs64')
learn.freeze()
# learn.summary()

class Flatten(nn.Module) :
    def forward(self, input):
        return input.view(input.size(0), -1)

def conv2(ni, nf) : 
    return conv_layer(ni, nf, stride = 2)

class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf,nf)
        
    def forward(self, x): 
        return (x + self.conv1(x))

def conv_and_res(ni, nf): 
    return nn.Sequential(conv2(ni, nf), ResBlock(nf))

def conv_(nf) : 
    return nn.Sequential(conv_layer(nf, nf), ResBlock(nf))
    
net = nn.Sequential(
    conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
    nn.MaxPool2d(3, 2, padding = 1),
    conv_(64),
    conv_and_res(64, 128),
    conv_and_res(128, 256),
    conv_and_res(256, 512),
    AdaptiveConcatPool2d(),
    Flatten(),
    nn.Linear(2 * 512, 256),
    nn.Linear(256, 10)
)

net.cpu()
if hyper_params['stage'] != 0 : 
    filename = '../saved_models/' + hyper_params['dataset'] + 'fsp_stage' + str(hyper_params['stage']) + '/model' + str(hyper_params['repeated']) + '.pt'
    net.load_state_dict(torch.load(filename, map_location = 'cpu'))

if torch.cuda.is_available() : 
    net = net.cuda()
    print('Model on GPU')
    
for name, param in net.named_parameters() : 
    print(name, param.shape)
    param.requires_grad = False
    if name[0] == str(hyper_params['stage'] + 1) and hyper_params['stage'] != 0 :
        param.requires_grad = True
    elif name[0] == str(hyper_params['stage']) and hyper_params['stage'] == 0 : 
        param.requires_grad = True
    print(param.requires_grad)

Model on GPU
0.0.weight torch.Size([64, 3, 7, 7])
True
0.2.weight torch.Size([64])
True
0.2.bias torch.Size([64])
True
2.0.0.weight torch.Size([64, 64, 3, 3])
False
2.0.2.weight torch.Size([64])
False
2.0.2.bias torch.Size([64])
False
2.1.conv1.0.weight torch.Size([64, 64, 3, 3])
False
2.1.conv1.2.weight torch.Size([64])
False
2.1.conv1.2.bias torch.Size([64])
False
3.0.0.weight torch.Size([128, 64, 3, 3])
False
3.0.2.weight torch.Size([128])
False
3.0.2.bias torch.Size([128])
False
3.1.conv1.0.weight torch.Size([128, 128, 3, 3])
False
3.1.conv1.2.weight torch.Size([128])
False
3.1.conv1.2.bias torch.Size([128])
False
4.0.0.weight torch.Size([256, 128, 3, 3])
False
4.0.2.weight torch.Size([256])
False
4.0.2.bias torch.Size([256])
False
4.1.conv1.0.weight torch.Size([256, 256, 3, 3])
False
4.1.conv1.2.weight torch.Size([256])
False
4.1.conv1.2.bias torch.Size([256])
False
5.0.0.weight torch.Size([512, 256, 3, 3])
False
5.0.2.weight torch.Size([512])
False
5.0.2.bias torch.Size([512])
Fa

In [22]:
# x, y = next(iter(data.train_dl))
# net(torch.autograd.Variable(x).cuda())
summary(net, (3, 224, 224))
# print(learn.summary())
learn.model
# net

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
              ReLU-2         [-1, 64, 112, 112]               0
       BatchNorm2d-3         [-1, 64, 112, 112]             128
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
              ReLU-6           [-1, 64, 56, 56]               0
       BatchNorm2d-7           [-1, 64, 56, 56]             128
            Conv2d-8           [-1, 64, 56, 56]          36,864
              ReLU-9           [-1, 64, 56, 56]               0
      BatchNorm2d-10           [-1, 64, 56, 56]             128
         ResBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12          [-1, 128, 28, 28]          73,728
             ReLU-13          [-1, 128, 28, 28]               0
      BatchNorm2d-14          [-1, 128,

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [5]:
class SaveFeatures :
    def __init__(self, m) : 
        self.handle = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, m, inp, outp) : 
        self.features = outp
    def remove(self) :
        self.handle.remove()
        
# saving outputs of all Basic Blocks
mdl = learn.model
sf = [SaveFeatures(m) for m in [mdl[0][3], mdl[0][4], mdl[0][5], mdl[0][5][0].downsample, mdl[0][6], mdl[0][6][0].downsample, mdl[0][7]]]
sf2 = [SaveFeatures(m) for m in [net[1], net[2], net[2], net[3], net[4], net[5]]]

In [6]:
# Testing working of SaveFeatures
x, y = next(iter(data.train_dl))
x = torch.autograd.Variable(x).cuda()
out1 = mdl(x)
out2 = net(x)
for i in range(len(sf)) : 
    print(i)
    print(sf[i].features.shape)
#     assert(sf[i].features.shape == sf[i + 1].features.shape)
#     assert(sf2[i].features.shape == sf2[i + 1].features.shape)
del x, y

0
torch.Size([64, 64, 56, 56])
1
torch.Size([64, 64, 56, 56])
2
torch.Size([64, 128, 28, 28])
3
torch.Size([64, 128, 28, 28])
4
torch.Size([64, 256, 14, 14])
5
torch.Size([64, 256, 14, 14])
6
torch.Size([64, 512, 7, 7])


In [17]:
# Testing working of FSP
x, y = next(iter(data.train_dl))
x = torch.autograd.Variable(x).cuda()
for i in range(4) : 
    _ = net(x)
    _ = mdl(x)
    fsp1 = fsp(sf[i].features, sf[i + 1].features)
    fsp2 = fsp(sf2[i].features, sf2[i + 1].features)
    print(F.mse_loss(fsp1, fsp2))

tensor(0.0716, device='cuda:0', grad_fn=<MeanBackward0>)


RuntimeError: size of dimension does not match previous size, operand 1, dim 1

#### Stage-wise training

In [ ]:
if hyper_params['stage'] == 0 : 
    filename = '../saved_models/smallest_stage' + str(hyper_params['stage']) + '/model' + str(hyper_params['repeated']) + '.pt'
else : 
    filename = '../saved_models/smallest_stage' + str(hyper_params['stage'] + 1) + '/model' + str(hyper_params['repeated']) + '.pt'
optimizer = torch.optim.Adam(net.parameters(), lr = hyper_params["learning_rate"])
total_step = len(data.train_ds) // hyper_params["batch_size"]
train_loss_list = list()
val_loss_list = list()
min_val = 100
for epoch in range(hyper_params["num_epochs"]):
    trn = []
    net.train()
    for i, (images, labels) in enumerate(data.train_dl) :
        if torch.cuda.is_available():
            images = torch.autograd.Variable(images).cuda().float()
            labels = torch.autograd.Variable(labels).cuda()
        else : 
            images = torch.autograd.Variable(images).float()
            labels = torch.autograd.Variable(labels)

        y_pred = net(images)
        y_pred2 = mdl(images)

        loss = F.mse_loss(sf2[hyper_params["stage"]].features, sf[hyper_params["stage"]].features)
        trn.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
#         torch.nn.utils.clip_grad_value_(net.parameters(), 10)
        optimizer.step()

        if i % 50 == 49 :
            print('epoch = ', epoch + 1, ' step = ', i + 1, ' of total steps ', total_step, ' loss = ', loss.item())

    train_loss = (sum(trn) / len(trn))
    train_loss_list.append(train_loss)

    net.eval()
    val = []
    with torch.no_grad() :
        for i, (images, labels) in enumerate(data.valid_dl) :
            if torch.cuda.is_available():
                images = torch.autograd.Variable(images).cuda().float()
                labels = torch.autograd.Variable(labels).cuda()
            else : 
                images = torch.autograd.Variable(images).float()
                labels = torch.autograd.Variable(labels)

            # Forward pass
            y_pred = net(images)
            y_pred2 = mdl(images)
            loss = F.mse_loss(sf[hyper_params["stage"]].features, sf2[hyper_params["stage"]].features)
            val.append(loss.item())

    val_loss = sum(val) / len(val)
    val_loss_list.append(val_loss)
    print('epoch : ', epoch + 1, ' / ', hyper_params["num_epochs"], ' | TL : ', train_loss, ' | VL : ', val_loss)
    experiment.log_metric("train_loss", train_loss)
    experiment.log_metric("val_loss", val_loss)

    if val_loss < min_val :
        print('saving model')
        min_val = val_loss
        torch.save(net.state_dict(), filename)

In [ ]:
# learn = Learner(data, net, metrics = accuracy)
net.cpu()
net.load_state_dict(torch.load('../saved_models/stage5/model0.pt', map_location = 'cpu'))
net = net.cuda()

In [ ]:
plt.plot(range(hyper_params["num_epochs"]), train_loss_list, label = 'train_loss') 
plt.plot(range(hyper_params["num_epochs"]), val_loss_list, label = 'val_loss')
plt.legend()
plt.savefig('../figures/stage5/train_loss.jpg')

#### Training the classifier only after stage-wise training

In [ ]:
hyper_params = {
    "stage": 5,
    "repeated": 3,
    "num_classes": 10,
    "batch_size": 64,
    "num_epochs": 100,
    "learning_rate": 1e-4
}

class Flatten(nn.Module) :
    def forward(self, input):
        return input.view(input.size(0), -1)

def conv2(ni, nf) : 
    return conv_layer(ni, nf, stride = 2)

class ResBlock(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv1 = conv_layer(nf,nf)
        
    def forward(self, x): 
        return (x + self.conv1(x))

def conv_and_res(ni, nf): 
    return nn.Sequential(conv2(ni, nf), ResBlock(nf))

def conv_(nf) : 
    return nn.Sequential(conv_layer(nf, nf), ResBlock(nf))
    
net = nn.Sequential(
    conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
    nn.MaxPool2d(3, 2, padding = 1),
    conv_(64),
    conv_and_res(64, 128),
    conv_and_res(128, 256),
    AdaptiveConcatPool2d(),
    Flatten(),
    nn.Linear(2 * 256, 128),
    nn.Linear(128, hyper_params["num_classes"])
)

net.cpu()
net.load_state_dict(torch.load('../saved_models/small_stage4/model1.pt', map_location = 'cpu'))

if torch.cuda.is_available() : 
    net = net.cuda()
    print('Model on GPU')
    
for name, param in net.named_parameters() : 
    print(name, param.shape)
    param.requires_grad = False
    if name[0] == '7' or name[0] == '8':
        param.requires_grad = True
    print(param.requires_grad)

In [ ]:
def _get_accuracy(dataloader, Net):
    total = 0
    correct = 0
    Net.eval()
    for i, (images, labels) in enumerate(dataloader):
        images = torch.autograd.Variable(images).float()
        labels = torch.autograd.Variable(labels).float()
        
        if torch.cuda.is_available() : 
            images = images.cuda()
            labels = labels.cuda()

        outputs = Net.forward(images)
        outputs = F.log_softmax(outputs, dim = 1)

        _, pred_ind = torch.max(outputs, 1)
        
        # converting to numpy arrays
        labels = labels.data.cpu().numpy()
        pred_ind = pred_ind.data.cpu().numpy()
        
        # get difference
        diff_ind = labels - pred_ind
        # correctly classified will be 1 and will get added
        # incorrectly classified will be 0
        correct += np.count_nonzero(diff_ind == 0)
        total += len(diff_ind)

    accuracy = correct / total
    # print(len(diff_ind))
    return accuracy

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr = hyper_params["learning_rate"])
total_step = len(data.train_ds) // hyper_params["batch_size"]
train_loss_list = list()
val_loss_list = list()
min_val = 0
for epoch in range(hyper_params["num_epochs"]):
    trn = []
    net.train()
    for i, (images, labels) in enumerate(data.train_dl) :
        if torch.cuda.is_available():
            images = torch.autograd.Variable(images).cuda().float()
            labels = torch.autograd.Variable(labels).cuda()
        else : 
            images = torch.autograd.Variable(images).float()
            labels = torch.autograd.Variable(labels)

        y_pred = net(images)

        loss = F.cross_entropy(y_pred, labels)
        trn.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
#         torch.nn.utils.clip_grad_value_(net.parameters(), 10)
        optimizer.step()

        if i % 50 == 49 :
            print('epoch = ', epoch, ' step = ', i + 1, ' of total steps ', total_step, ' loss = ', loss.item())

    train_loss = (sum(trn) / len(trn))
    train_loss_list.append(train_loss)

    net.eval()
    val = []
    with torch.no_grad() :
        for i, (images, labels) in enumerate(data.valid_dl) :
            if torch.cuda.is_available():
                images = torch.autograd.Variable(images).cuda().float()
                labels = torch.autograd.Variable(labels).cuda()
            else : 
                images = torch.autograd.Variable(images).float()
                labels = torch.autograd.Variable(labels)

            # Forward pass
            y_pred = net(images)
            
            loss = F.cross_entropy(y_pred, labels)
            val.append(loss.item())

    val_loss = sum(val) / len(val)
    val_loss_list.append(val_loss)
    val_acc = _get_accuracy(data.valid_dl, net)

    print('epoch : ', epoch + 1, ' / ', hyper_params["num_epochs"], ' | TL : ', train_loss, ' | VL : ', val_loss, ' | VA : ', val_acc * 100)

    if (val_acc * 100) > min_val :
        print('saving model')
        min_val = val_acc * 100
        torch.save(net.state_dict(), '../saved_models/small_classifier/model1.pt')savename

In [ ]:
net.cpu()
net.load_state_dict(torch.load('../saved_models/small_classifier/model4.pt', map_location = 'cpu'))
net.cuda()

learn = cnn_learner(data, models.resnet34, metrics = accuracy)
learn = learn.load('unfreeze_imagenet_bs64')
learn.freeze()

print(_get_accuracy(data.valid_dl, net))
print(_get_accuracy(data.valid_dl, learn.model))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from fastai.vision import *
import torch
from torchsummary import summary
torch.cuda.set_device(0)

for repeated in range(0, 1) : 
    torch.manual_seed(repeated)
    torch.cuda.manual_seed(repeated)

    # stage should be in 0 to 5 (5 for classifier stage)
    hyper_params = {
        "stage": 5,
        "repeated": repeated,
        "num_classes": 10,
        "batch_size": 64,
        "num_epochs": 100,
        "learning_rate": 1e-4
    }

    path = untar_data(URLs.IMAGENETTE)
    tfms = get_transforms(do_flip=False)
    data = ImageDataBunch.from_folder(path, train = 'train', valid = 'val', bs = hyper_params["batch_size"], size = 224, ds_tfms = tfms).normalize(imagenet_stats)
    
    class Flatten(nn.Module) :
        def forward(self, input):
            return input.view(input.size(0), -1)

    def conv2(ni, nf) : 
        return conv_layer(ni, nf, stride = 2)

    class ResBlock(nn.Module):
        def __init__(self, nf):
            super().__init__()
            self.conv1 = conv_layer(nf,nf)

        def forward(self, x): 
            return (x + self.conv1(x))

    def conv_and_res(ni, nf): 
        return nn.Sequential(conv2(ni, nf), ResBlock(nf))

    def conv_(nf) : 
        return nn.Sequential(conv_layer(nf, nf), ResBlock(nf))

    net = nn.Sequential(
        conv_layer(3, 64, ks = 7, stride = 2, padding = 3),
        nn.MaxPool2d(3, 2, padding = 1),
        conv_(64),
        conv_and_res(64, 128),
        conv_and_res(128, 256),
        AdaptiveConcatPool2d(),
        Flatten(),
        nn.Linear(2 * 256, 128),
        nn.Linear(128, hyper_params["num_classes"])
    )

    net.cpu()
    filename = '../saved_models/small_stage4/model' + str(repeated) + '.pt'
    net.load_state_dict(torch.load(filename, map_location = 'cpu'))

    if torch.cuda.is_available() : 
        net = net.cuda()
        print('Model on GPU')

    for name, param in net.named_parameters() : 
        param.requires_grad = False
        if name[0] == '7' or name[0] == '8':
            param.requires_grad = True
        
    optimizer = torch.optim.Adam(net.parameters(), lr = hyper_params["learning_rate"])
    total_step = len(data.train_ds) // hyper_params["batch_size"]
    train_loss_list = list()
    val_loss_list = list()
    min_val = 0
    savename = '../saved_models/small_classifier/model' + str(repeated) + '.pt'
    for epoch in range(hyper_params["num_epochs"]):
        trn = []
        net.train()
        for i, (images, labels) in enumerate(data.train_dl) :
            if torch.cuda.is_available():
                images = torch.autograd.Variable(images).cuda().float()
                labels = torch.autograd.Variable(labels).cuda()
            else : 
                images = torch.autograd.Variable(images).float()
                labels = torch.autograd.Variable(labels)

            y_pred = net(images)

            loss = F.cross_entropy(y_pred, labels)
            trn.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
    #         torch.nn.utils.clip_grad_value_(net.parameters(), 10)
            optimizer.step()

            if i % 50 == 49 :
                print('epoch = ', epoch, ' step = ', i + 1, ' of total steps ', total_step, ' loss = ', loss.item())

        train_loss = (sum(trn) / len(trn))
        train_loss_list.append(train_loss)

        net.eval()
        val = []
        with torch.no_grad() :
            for i, (images, labels) in enumerate(data.valid_dl) :
                if torch.cuda.is_available():
                    images = torch.autograd.Variable(images).cuda().float()
                    labels = torch.autograd.Variable(labels).cuda()
                else : 
                    images = torch.autograd.Variable(images).float()
                    labels = torch.autograd.Variable(labels)

                # Forward pass
                y_pred = net(images)

                loss = F.cross_entropy(y_pred, labels)
                val.append(loss.item())

        val_loss = sum(val) / len(val)
        val_loss_list.append(val_loss)
        val_acc = _get_accuracy(data.valid_dl, net)

        print('epoch : ', epoch + 1, ' / ', hyper_params["num_epochs"], ' | TL : ', train_loss, ' | VL : ', val_loss, ' | VA : ', val_acc * 100)

        if (val_acc * 100) > min_val :
            print('saving model')
            min_val = val_acc * 100
            torch.save(net.state_dict(), savename)